<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=acb1dba3556ad4c29c7a3c0c3a83bab98d6a2d0b60d69abdaed62a36e62008b4
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 09:16:55
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -9.65 K (-0.07%)
Current PnL: -20.55 L (-13.69%)
CY Booked + Current PnL: -9.11 L (-6.07%)
-------------------
Total profit:  1.73 L
Total loss:  -22.28 L
-------------------
Total Booked + Current PnL: 17.87 L (14.52%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.11%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.05 L (62.43%)
Deployed:  1.23 C
Current:  1.41 C
CAGR/XIRR %: 8.17%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,72.0,H-MC,14.65,121662.0,21158.0,7945.0,2.65,21.05,6.53,28.96,79.0,2.66,0.87,62.17,MH,ATH,METALS
14,BLUESTARCO,2080.00,13.31,60.0,X-MC,3.24,195690.0,31020.0,12309.0,3.41,18.84,6.29,26.31,89.0,2.52,1.39,28.32,X40N,NTT,AC
77,TTKPRESTIG,770.00,92.96,39.0,M-SC,1.01,83093.0,-17684.0,17774.0,-0.43,-17.55,21.39,0.09,245.0,-0.99,0.59,9.17,OX40N,NTT,DURABLES
84,VOLTAS,1530.00,0.60,59.0,X-MC,1.07,211200.0,19458.0,18290.0,1.88,10.15,8.66,19.69,99.0,1.06,1.50,17.64,XY25,NTT,AC
19,CIPLA,1662.74,-22.52,44.0,X-LC,3.61,206774.0,2274.0,21029.0,-0.25,1.11,10.17,11.39,10.0,0.11,1.47,10.16,X40N,ATH,PHARMA


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
46,KPIGREEN,731.05,-7.49,38.0,H-SC,6.26,110275.0,-15022.0,73950.0,3.75,-11.99,67.06,47.03,141.0,-0.20,0.79,38.07,MH,ATH,POWER
14,BLUESTARCO,2080.00,13.31,60.0,X-MC,3.24,195690.0,31020.0,12309.0,3.41,18.84,6.29,26.31,89.0,2.52,1.39,28.32,X40N,NTT,AC
50,NATIONALUM,244.55,-38.69,72.0,H-MC,14.65,121662.0,21158.0,7945.0,2.65,21.05,6.53,28.96,79.0,2.66,0.87,62.17,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-48.11,32.0,M-SC,2.43,87600.0,-3898.0,82651.0,2.09,-4.26,94.35,86.07,223.0,-0.05,0.62,27.21,XR,NTT,DURABLES
84,VOLTAS,1530.00,0.60,59.0,X-MC,1.07,211200.0,19458.0,18290.0,1.88,10.15,8.66,19.69,99.0,1.06,1.50,17.64,XY25,NTT,AC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,103.12,27.0,H-SC,7.91,137696.0,-80800.0,108986.0,-4.13,-36.98,79.15,12.90,153.0,-0.74,0.98,4.81,OX40N,NTT,FINANCE
32,HINDZINC,730.22,32.99,64.0,M-LC,7.79,216543.0,11467.0,101104.0,-2.82,5.59,46.69,54.89,52.0,0.11,1.54,31.03,X5K,ATH,METALS
72,TATAELXSI,9161.00,-23.21,49.0,H-MC,6.61,103940.0,-27895.0,70118.0,-1.84,-21.16,67.46,32.03,98.0,-0.40,0.74,16.42,OX40N,NTT,IT
61,SAMMAANCAP,326.00,-182.95,62.0,M-SC,15.26,143298.0,-5922.0,150105.0,-1.14,-3.97,104.75,96.62,208.0,-0.04,1.02,55.08,XY25,NTT,FINANCE
37,INDIAMART,4810.62,-54.64,41.0,H-SC,8.72,126410.0,3074.0,128546.0,-1.12,2.49,101.69,106.72,119.0,0.02,0.90,26.45,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,106.53,61.0,H-SC,13.72,130140.0,-12339.0,31858.0,-0.02,-8.66,24.48,13.70,163.0,-0.39,0.93,53.60,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,92.96,39.0,M-SC,1.01,83093.0,-17684.0,17774.0,-0.43,-17.55,21.39,0.09,245.0,-0.99,0.59,9.17,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,48.0,H-SC,1.92,220734.0,-48933.0,85270.0,-0.04,-18.15,38.63,13.47,138.0,-0.57,1.57,12.24,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.21,49.0,H-MC,6.61,103940.0,-27895.0,70118.0,-1.84,-21.16,67.46,32.03,98.0,-0.40,0.74,16.42,OX40N,NTT,IT
18,CERA,9475.0,-24.34,34.0,H-SC,1.80,137839.0,-38064.0,80084.0,-0.50,-21.64,58.10,23.89,149.0,-0.48,0.98,19.04,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,72.0,H-MC,14.65,121662.0,21158.0,7945.0,2.65,21.05,6.53,28.96,79.0,2.66,0.87,62.17,MH,ATH,METALS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,48.36,59.0,M-MC,12.81,233675.0,8713.0,158946.0,-0.56,3.87,68.02,74.53,192.0,0.05,1.66,37.38,XY24,BTT,STEEL
32,HINDZINC,730.22,32.99,64.0,M-LC,7.79,216543.0,11467.0,101104.0,-2.82,5.59,46.69,54.89,52.0,0.11,1.54,31.03,X5K,ATH,METALS
86,WIPRO,420.00,-12.89,51.0,M-LC,5.65,152768.0,1823.0,107625.0,0.87,1.21,70.45,72.51,53.0,0.02,1.09,7.30,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,54.0,M-LC,9.12,156679.0,15839.0,28657.0,-0.02,11.25,18.29,31.59,66.0,0.55,1.12,40.71,XY24,NTT,BANKS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,11.91,193757.0,13949.0,98467.0,0.15,7.76,50.82,62.52,88.0,0.14,1.38,44.49,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-38.69,72.0,H-MC,14.65,121662.0,21158.0,7945.0,2.65,21.05,6.53,28.96,79.0,2.66,0.87,62.17,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,11.91,193757.0,13949.0,98467.0,0.15,7.76,50.82,62.52,88.0,0.14,1.38,44.49,XR,NTT,BANKS
32,HINDZINC,730.22,32.99,64.0,M-LC,7.79,216543.0,11467.0,101104.0,-2.82,5.59,46.69,54.89,52.0,0.11,1.54,31.03,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.64,41.0,H-SC,8.72,126410.0,3074.0,128546.0,-1.12,2.49,101.69,106.72,119.0,0.02,0.90,26.45,AR,ATH,MISC
86,WIPRO,420.00,-12.89,51.0,M-LC,5.65,152768.0,1823.0,107625.0,0.87,1.21,70.45,72.51,53.0,0.02,1.09,7.30,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,26.0,X-SC,37.84,52144.0,-12862.0,163790.0,0.04,-19.79,314.11,232.18,198.0,-0.08,0.37,0.04,XY24,NTT,MISC
23,DABUR,735.00,-12.66,33.0,X-MC,3.74,236244.0,-11824.0,120225.0,0.35,-4.77,50.89,43.70,102.0,-0.10,1.68,8.99,XY24,BTT,FMCG
29,HAVELLS,2069.16,-6.75,33.0,X-MC,0.62,231098.0,-16633.0,91700.0,-0.30,-6.71,39.68,30.30,92.0,-0.18,1.65,5.69,X40,ATH,ELECTRICAL
22,COLPAL,3767.14,-5.64,34.0,X-MC,7.31,220440.0,-42925.0,156270.0,-0.19,-16.30,70.89,43.04,84.0,-0.27,1.57,2.34,XY25,ATH,FMCG
5,ASIANPAINT,3465.66,-18.16,35.0,X-LC,5.55,206536.0,-45232.0,98435.0,0.45,-17.97,47.66,21.13,27.0,-0.46,1.47,11.09,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-6.75,33.0,X-MC,0.62,231098.0,-16633.0,91700.0,-0.30,-6.71,39.68,30.30,92.0,-0.18,1.65,5.69,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,0.60,59.0,X-MC,1.07,211200.0,19458.0,18290.0,1.88,10.15,8.66,19.69,99.0,1.06,1.50,17.64,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.16,44.0,X-LC,1.59,247022.0,-29501.0,139567.0,-0.09,-10.67,56.50,39.81,54.0,-0.21,1.76,18.39,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.33,48.0,H-LC,1.68,158550.0,-27545.0,75026.0,-0.23,-14.80,47.32,25.51,15.0,-0.37,1.13,16.84,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-18.72,49.0,X-MC,1.87,139712.0,3579.0,27551.0,0.28,2.63,19.72,22.86,91.0,0.13,0.99,15.57,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,26.0,X-SC,37.84,52144.0,-12862.0,163790.0,0.04,-19.79,314.11,232.18,198.0,-0.08,0.37,0.04,XY24,NTT,MISC
83,VBL,671.64,-21.40,39.0,X-LC,10.36,283272.0,-32570.0,145234.0,-0.43,-10.31,51.27,35.67,5.0,-0.22,2.02,2.11,X40N,ATH,FMCG
22,COLPAL,3767.14,-5.64,34.0,X-MC,7.31,220440.0,-42925.0,156270.0,-0.19,-16.30,70.89,43.04,84.0,-0.27,1.57,2.34,XY25,ATH,FMCG
42,ITC,452.00,-39.61,40.0,X-LC,2.44,195478.0,-4660.0,25549.0,-0.04,-2.33,13.07,10.44,4.0,-0.18,1.39,3.11,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.06,40.0,X-MC,4.94,180908.0,-19598.0,116758.0,0.20,-9.77,64.54,48.45,90.0,-0.17,1.29,4.32,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-47.66,26.0,X-SC,37.84,52144.0,-12862.0,163790.0,0.04,-19.79,314.11,232.18,198.0,-0.08,0.37,0.04,XY24,NTT,MISC
56,RELAXO,1176.00,-40.07,38.0,X-SC,2.56,81630.0,-63530.0,139457.0,-0.84,-43.77,170.84,52.31,136.0,-0.46,0.58,9.27,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-34.87,44.0,X-SC,4.42,97434.0,-31236.0,76534.0,0.14,-24.28,78.55,35.20,219.0,-0.41,0.69,11.83,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-5.16,50.0,X-SC,1.45,107884.0,5880.0,38040.0,0.57,5.76,35.26,43.06,122.0,0.15,0.77,23.87,X40N,ATH,MISC
33,HONAUT,58357.33,-23.00,45.0,X-SC,2.16,108540.0,-19398.0,66535.0,-0.17,-15.16,61.30,36.84,143.0,-0.29,0.77,11.12,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.36,52.0,X-LC,12.15,285581.0,-60375.0,128626.0,-0.77,-17.45,45.04,19.73,1.0,-0.47,2.03,5.18,X40,ATH,IT
40,INFY,2275.00,-17.80,56.0,X-LC,7.79,320735.0,7847.0,163831.0,-0.23,2.51,51.08,54.87,3.0,0.05,2.28,9.32,X40,BTT,IT
42,ITC,452.00,-39.61,40.0,X-LC,2.44,195478.0,-4660.0,25549.0,-0.04,-2.33,13.07,10.44,4.0,-0.18,1.39,3.11,X40,NTT,FMCG
83,VBL,671.64,-21.40,39.0,X-LC,10.36,283272.0,-32570.0,145234.0,-0.43,-10.31,51.27,35.67,5.0,-0.22,2.02,2.11,X40N,ATH,FMCG
1,ABB,7934.00,-40.84,43.0,H-LC,7.51,247440.0,-14179.0,133395.0,0.17,-5.42,53.91,45.57,7.0,-0.11,1.76,5.29,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,50.0,L-SC,29.06,81408.0,-32141.0,72193.0,-0.70,-28.31,88.68,35.27,268.0,-0.45,0.58,99.25,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,42.0,H-SC,10.36,90630.0,-7350.0,28956.0,0.30,-7.50,31.95,22.05,152.0,-0.25,0.65,31.74,XR,ATH,FINANCE
50,NATIONALUM,244.55,-38.69,72.0,H-MC,14.65,121662.0,21158.0,7945.0,2.65,21.05,6.53,28.96,79.0,2.66,0.87,62.17,MH,ATH,METALS
78,UJJIVANSFB,60.00,106.53,61.0,H-SC,13.72,130140.0,-12339.0,31858.0,-0.02,-8.66,24.48,13.70,163.0,-0.39,0.93,53.60,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,62.0,M-SC,15.26,143298.0,-5922.0,150105.0,-1.14,-3.97,104.75,96.62,208.0,-0.04,1.02,55.08,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,64.0,H-SC,8.96,151317.0,-27754.0,93484.0,-0.17,-15.50,61.78,36.71,135.0,-0.30,1.08,50.14,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-182.95,62.0,M-SC,15.26,143298.0,-5922.0,150105.0,-1.14,-3.97,104.75,96.62,208.0,-0.04,1.02,55.08,XY25,NTT,FINANCE
32,HINDZINC,730.22,32.99,64.0,M-LC,7.79,216543.0,11467.0,101104.0,-2.82,5.59,46.69,54.89,52.0,0.11,1.54,31.03,X5K,ATH,METALS
26,GREENPANEL,537.00,231.40,58.0,M-SC,4.66,147962.0,-35116.0,114093.0,-0.61,-19.18,77.11,43.14,230.0,-0.31,1.05,39.18,XY24,NTT,MISC
50,NATIONALUM,244.55,-38.69,72.0,H-MC,14.65,121662.0,21158.0,7945.0,2.65,21.05,6.53,28.96,79.0,2.66,0.87,62.17,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.16
1,25,43.45
2,50,75.14


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.92
LC    32.25
MC    23.80
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.96
X40      14.61
X40N     11.24
XR       10.96
XY25     10.39
AR        8.17
OX40N     7.60
X200      1.73
MH        1.66
X5K       1.54
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.50
X-LC    22.87
X-MC    17.20
M-SC    12.49
H-MC     4.64
H-LC     4.62
X-SC     4.50
M-LC     3.75
M-MC     1.66
L-SC     1.43
L-LC     1.01
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.73,-6.36,41.41
IT,12.85,-16.39,79.43
FINANCE,10.44,-15.09,70.66
MISC,6.89,-18.70,84.47
BANKS,6.38,-11.56,70.25
PAINTS,5.50,-19.37,37.16
ELECTRICAL,5.31,-10.43,49.17
INSURANCE,3.81,-3.33,41.95
AC,3.58,3.94,12.14


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3225292.0,22
XR,1370289.0,14
AR,1182972.0,9
X40,785993.0,10
X40N,655937.0,9
OX40N,577684.0,10
XY25,476555.0,7
SR,265561.0,2
X5K,101104.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3422651.0,25
M-SC,1283805.0,15
X-MC,1117293.0,12
X-LC,1026718.0,13
X-SC,547460.0,6
H-MC,358856.0,4
H-LC,290392.0,3
L-SC,259826.0,3
M-LC,237386.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1199964.0      6
           AR         808900.0      5
           XR         801738.0      7
M-SC       XY24       690271.0      6
X-MC       XY24       593369.0      4
X-LC       X40        489162.0      5
X-SC       X40N       307136.0      4
H-SC       OX40N      272538.0      4
           SR         265561.0      2
X-MC       X40        220297.0      4
X-LC       X40N       219734.0      3
H-LC       AR         208421.0      2
X-LC       XY24       207969.0      2
H-MC       XY24       182326.0      1
X-MC       XY25       174560.0      2
L-SC       XR         170892.0      2
M-SC       AR         165651.0      2
X-SC       XY24       163790.0      1
M-MC       XY24       158946.0      1
M-SC       XY25       150105.0      1
           OX40N      146094.0      4
           XR         131684.0      2
X-MC       X40N       129067.0      2
X-LC       XY25       109853.0      3
M-LC       XR         107625.0      1
           X5K        101104.0      1
H-MC       XR          98467.0      1
L-SC       OX40N       88934.0      1
H-LC       X200        81971.0      1
X-SC       X40         76534.0      1
H-SC       MH          73950.0      1
H-MC       OX40N       70118.0      1
L-MC       XR          59883.0      1
L-LC       XY25        42037.0      1
M-LC       XY24        28657.0      1
H-MC       MH           7945.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
